In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import functools
import os
import pandas as pd
import re

from IPython.display import display

from cryptic.constants import BASEDIR

os.chdir(BASEDIR)

from cryptic.datasets.puzzle import QAFrame
from cryptic.models.oai import OpenAIQA
from cryptic.models.selectors import SelfConsistencySelector
from cryptic.models.validators import SubWordConsistencyValidator
from cryptic.prompts.interface import PromptInterface

pd.set_option('max_colwidth', 500)

# Observations

1. It's better at short clues:

        Heads of South Korean institute go downhill fast - SKI
        Oddly envies the first lady - EVE

2. It comes up with a wordplay to suit the answer
     - we could also try the prompt the other way around and average

3. Anagram and repeated extraction indicators really throw it.

# TODO

1. Multiprocess requests -- see OpenAI example
2. Embeddings validator of all components
3. Make extraction of answer not conditional on matching a regexp for the wordplay
4. Prompt to decompose first, then generate solutions and sub answers conditioned on decomposition.

## For interactive tests of prompt injection, answer passing and API output see bottom of NB

In [5]:
PUZZLE_ID = 1223
NUM_CLUES = None
NUM_ANSWERS = 10

In [6]:
prompt_interface = PromptInterface("cryptic.prompts.cot.ben_v1")
validator = SubWordConsistencyValidator(prompt_interface)

In [7]:
puzzle = QAFrame.from_quiptic_json(f"data/quiptics/puzzles/{PUZZLE_ID}.json")

In [8]:
model = OpenAIQA(
    prompt_interface=prompt_interface,
    model_name="text-davinci-003",
    max_tokens=64,
    validators=[validator],
    validator_names=["consistent_decomposition"]
)

In [9]:
df = puzzle.df
df.head(3)

,clue,id,num_letters,code,orientation,rowid
0,"Leo possibly seeing good chieftains travelling around end of Suez (4,2,3,6)",1,"(4, 2, 3, 6)",1A,across,0
1,Owning commercial US college can gratify leader (9),9,9,3A,across,1
2,"Quickly visit Post Office to get code for debit card (3,2)",10,"(3, 2)",3K,across,2


In [10]:
answer_df = model.predict(puzzle.df, num_answers=10)

0it [00:00, ?it/s]


AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

In [36]:
cols = [f"reconstruction-{i}" for i in range(1)]
pred_cols = [f"prediction-{i}" for i in range(1)]

In [37]:
selector = SelfConsistencySelector(model.validator_names)
long_df = selector.make_long_df(answer_df)

In [38]:
candidates = long_df[~(long_df["prediction"]=="")&(long_df["prediction_satisfies_constraints"])]
for ix, row in answer_df[~(answer_df[pred_cols] == "").all(axis=1)].iterrows():
    row_cand = candidates[candidates["clue"]==row["clue"]].reset_index()[["reconstruction"]]
    print(row["clue"])
    if len(row_cand) > 0:
        display(row_cand)
    print("\n---------\n")

Owning commercial US college can gratify leader (9)


,reconstruction
0,PROFITING (owning commercial) - US (United States) + OF (college) + IT (can) + ING (gratify leader)



---------

Group support for an unfortunate event (7)

---------

Agent and setter returning alternative ruler (7)


,reconstruction
0,REPLACE (alternative ruler) - RE (agent) + PLACE (setter)
1,REPLACE (alternative ruler) - RE (agent) + PLACE (setter)
2,REPLACE (alternative ruler) - RE (agent) + PLACE (setter returning)
3,REPLACE (alternative ruler) - RE (agent) + PLACE (setter returning)
4,REPLACE (alternative ruler) - RE (agent) + PLACE (setter returning)
5,REPLACE (alternative ruler) - RE (agent) + PLACE (setter)
6,REPLACE (alternative ruler) - RE (agent) + PLACE (setter returning)
7,REPLACE (alternative ruler) - RE (agent) + PLACE (setter returning)



---------

A fellow repeatedly found in home is most uncool (7)

---------

Employee in rowdy taverns (7)


...


---------

Film following flyer sent up and left to drop (7)


,reconstruction
0,AIRDROP (film following flyer sent up and left to drop) - AIR (flyer sent up) + DROP (left to drop)
1,AIRDROP (film following flyer sent up and left to drop) - AIR (following flyer) + DROP (sent up and left).
2,AIRDROP (film following flyer sent up and left to drop) - AIR (following flyer) + DROP (sent up and left)



---------

Great secretary is a relative (7)


,reconstruction
0,COUSINS (relative) - COUSIN (great secretary) + S (is)
1,UNCLEAR (great) - UNCLE (relative) + AR (secretary)
2,COUSINS (relative) - COUSIN (great secretary) + S (is)
3,COUSINS (relative) - COUSIN (great secretary) + S (is)
4,UNCLEAR (great) - UNCLE (relative) + AR (secretary)
5,COUSINS (relative) - COUSIN (great secretary) + S (is)
6,COUSINS (relative) - COUSIN (great secretary) + S (is)



---------

Controls regarding home trespass (5,2)

---------



In [14]:
selector.select(answer_df)

,rowid,index,clue,code,orientation,reconstruction-0,num_letters,id,prediction,predicted_definition,wordplay,response,consistent_decomposition,prediction_satisfies_constraints
0,6,0,A fellow repeatedly found in home is most uncool (7),7A,across,UNHIPPE (most uncool) - UN (repeatedly) + HIP (found in home) + PE (fellow),7,14,UNHIPPE,most uncool,UN (repeatedly) + HIP (found in home) + PE (fellow),\nAnswer: UNHIPPE (most uncool) - UN (repeatedly) + HIP (found in home) + PE (fellow),True,True
1,7,0,Employee in rowdy taverns (7),7I,across,BARMAID (employee in rowdy taverns) - BAR (rowdy taverns) + MAID (employee),7,17,BARMAID,employee in rowdy taverns,BAR (rowdy taverns) + MAID (employee),\nAnswer: BARMAID (employee in rowdy taverns) - BAR (rowdy taverns) + MAID (employee),True,True
2,8,0,Celebration of son getting back ending finally after one (7),9A,across,REUNION (celebration) - RE (son getting back) + UNION (ending finally after one),7,19,REUNION,celebration,RE (son getting back) + UNION (ending finally after one),\nAnswer: REUNION (celebration) - RE (son getting back) + UNION (ending finally after one),True,True
3,9,0,Revolting jerk taking part in genuine performance (7),9I,across,REENACT (taking part in a genuine performance) - RE (revolting) + ENACT (jerk),7,22,REENACT,taking part in a genuine performance,RE (revolting) + ENACT (jerk),\nAnswer: REENACT (taking part in a genuine performance) - RE (revolting) + ENACT (jerk),True,True
4,13,0,Fruit left for gem of a girl (5),13A,across,"APRIL (fruit) - AP (left) + RIL (gem of a girl, i.e. April)",5,28,APRIL,fruit,"AP (left) + RIL (gem of a girl, i.e. April)","\nAnswer: APRIL (fruit) - AP (left) + RIL (gem of a girl, i.e. April)",True,True
5,26,0,Oddly envies the first lady (3),7J,down,EVE (envies) - E (first) + VE (lady),3,18,EVE,envies,E (first) + VE (lady),\nAnswer: EVE (envies) - E (first) + VE (lady),True,True


# Checks of prompt injection, answer extraction, API

In [16]:
DUMMY_CLUE = "A farmer's son"

prompt = prompt_interface.inject_prompt(DUMMY_CLUE, 5)
print(prompt)

out = prompt_interface.prompt_templates["example_output"]
print(out)

res = prompt_interface.extract_answer(out)
print(res)

prompt_interface.decompose(res["wordplay"])

I will answer some clues:
    1. Boy captures the work of Shakespeare, for example (7)
    Answer: SONNETS (examples of works of Shakespeare) - SON (boy) + NETS (captures)
    2. Country folk appearing in operatic piece (7)
    Answer: ARMENIA (country) - ARIA (operatic piece) + MEN (folk)
    3. Where men are on board ship beside revolutionary (5)
    Answer: CHESS (where men are on board) - CHE (Revolutionary, Che Guevara) + SS (designation for a ship)
    4. Amputate limbs to remove weapons (6)
    Answer: DISARM (Remove weapons  - DIS-ARM (amputating limbs)
    5. Accuse one politician apiece (7)
    Answer: IMPEACH (accuse) - I (one) + MP (politician) + EACH (apiece)
    6. Orchestra was taken aback by cutting machine (7)
    Answer: BANDSAW (cutting machine) - BAND (orchestra) + SAW (was, 'taken aback' i.e. backwards)
    7. Concerning drama in second game after draw (6)
    Answer: REPLAY (a second game, typically following a drawn result in the first) - RE (meaning concerning) 

[{'answer': 'RE', 'definition': 'meaning concerning'},
 {'answer': 'PLAY', 'definition': 'a type of drama'}]

## API Interaction

In [12]:
clue, num_letters = puzzle.sample(1).df.iloc[0][["clue", "num_letters"]]

response = model.get_response(clue, num_letters, num_answers=2)
print(response)

out = response["choices"][0]["text"]

res = prompt_interface.extract_answer(response["choices"][0]["text"])
print(res)

print(prompt_interface.decompose(res["wordplay"]))

validator.validate(res["answer"], res["predicted_definition"], res["wordplay"])